In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path
import os

In [7]:
cwd = Path.cwd().parent.parent
root_dir = Path( cwd / "01-intro")
def read_data(data_path:str) -> pd.DataFrame:
    data_path = Path(data_path)
    data_dir = root_dir / data_path
    print(data_dir.resolve())
    data01 = pd.read_parquet(data_dir)
    
    return data01

dataset = read_data("assignment/data/yellow_tripdata_2023-01.parquet")
dataset.head()

/Users/alghali/Downloads/AI-Compeitions/mlops-zoomcamp/01-intro/assignment/data/yellow_tripdata_2023-01.parquet


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [9]:
len(dataset.columns.tolist())

19

In [ ]:

dataset['tpep_pickup_datetime'] = pd.to_datetime(dataset["tpep_pickup_datetime"])
dataset['tpep_dropoff_datetime'] = pd.to_datetime(dataset['tpep_dropoff_datetime'])
    
dataset['pickup minutes'] = dataset['tpep_pickup_datetime'].dt.minute
dataset['drop minutes'] = dataset['tpep_dropoff_datetime'].dt.minute

dataset['pickup hour'] = dataset['tpep_pickup_datetime'].dt.hour 
dataset['drop hour'] = dataset['tpep_dropoff_datetime'].dt.hour 

dataset['difference in hours'] = abs(dataset['pickup hour'] - dataset['drop hour'])
dataset['min from hour'] = dataset['difference in hours'] * 60; 
dataset['duraion'] = abs(dataset['drop minutes'] - dataset['pickup minutes']) + dataset['min from hour']


In [18]:
dataset['tpep_pickup_datetime'] = pd.to_datetime(dataset['tpep_pickup_datetime'])
dataset['tpep_dropoff_datetime'] = pd.to_datetime(dataset['tpep_dropoff_datetime'])

dataset['duration'] = (dataset['tpep_dropoff_datetime'] - dataset['tpep_pickup_datetime']).dt.total_seconds() / 60

dataset.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,airport_fee,pickup minutes,drop minutes,pickup hour,drop hour,difference in hours,min form hour,min from hour,duraion,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,0.00,32,40,0,0,0,0,0,8,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,0.00,55,1,0,1,1,60,60,114,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,0.00,25,37,0,0,0,0,0,12,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,1.25,3,13,0,0,0,0,0,10,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,0.00,10,21,0,0,0,0,0,11,10.833333


In [19]:
dataset['duration'].std()

42.594351241920904

### Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

90%

92%

95%

98%

In [23]:
dataset.shape[0]

3066766

In [26]:
dataset_filtered = dataset[(dataset['duration'] >= 1) & (dataset['duration'] <= 60)]
ratio = dataset_filtered.shape[0] / dataset.shape[0]
ratio

0.9812202822125979